#### Задача 1 (2 балла).

Возьмите два больших текста (на любом языке, но оба на одном; это могут быть художественные тексты двух разных писателей, или тексты разного жанра). Посчитайте для каждого текста статистику по частям речи. Похожими ли получились цифры?

In [ ]:
!pip install spacy

In [ ]:
import spacy
from collections import Counter

In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
with open('/content/Иван Тургенев - Три встречи.txt', encoding = 'cp1251') as f:
    turgenev = f.read()
with open('/content/Антон Чехов - Каштанка.txt', encoding = 'utf8') as f:
    checkov = f.read()

In [ ]:
def pos_counter(text):
    nlp = spacy.load("ru_core_news_sm")
    doc = nlp(text)

    all_pos = [token.pos_ for token in doc] # соберем в список части речи всех слов
    statistics_dict = Counter(all_pos) # посчитаем статистику

    return dict(statistics_dict)

In [ ]:
turgenev_dict = pos_counter(turgenev)
turgenev_dict

{'PROPN': 177,
 'SPACE': 295,
 'NUM': 106,
 'NOUN': 1579,
 'PUNCT': 2703,
 'ADV': 770,
 'PRON': 1205,
 'VERB': 1574,
 'PART': 444,
 'ADP': 872,
 'SCONJ': 186,
 'DET': 390,
 'ADJ': 713,
 'CCONJ': 442,
 'X': 40,
 'AUX': 91,
 'INTJ': 3}

In [ ]:
checkov_dict = pos_counter(checkov)
checkov_dict

{'NOUN': 1374,
 'SPACE': 165,
 'ADJ': 418,
 'PUNCT': 1440,
 'ADP': 663,
 'ADV': 485,
 'VERB': 1171,
 'CCONJ': 471,
 'PRON': 550,
 'SCONJ': 136,
 'NUM': 52,
 'DET': 183,
 'PROPN': 251,
 'AUX': 70,
 'PART': 194,
 'INTJ': 10,
 'X': 1}

In [ ]:
# сделаем одинаковый порядок ключей

parts_of_speech = sorted(set(turgenev_dict.keys()).union(set(checkov_dict.keys())))

turgenev_stats = [turgenev_dict.get(pos, 0) for pos in parts_of_speech]
checkov_stats = [checkov_dict.get(pos, 0) for pos in parts_of_speech]

In [ ]:
from scipy.stats import spearmanr

In [ ]:
spearmanr(turgenev_stats, checkov_stats)

SignificanceResult(statistic=0.9583333333333335, pvalue=1.4469911897074028e-09)

In [ ]:
# коэффициент Спирмена близок к 1, значит, распределение частей речи в текстах похоже

#### Задача 2.

Возьмите датасет "Титаник" из 4 семинара в прошлом семестре и удостоверьтесь, влияет ли класс, которым плыл пассажир, на его шанс выжить (scipy умеет работать с датафреймами).

In [30]:
import pandas as pd

In [32]:
titanic = pd.read_csv('/content/titanic (1).csv', sep=';')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Braund; Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Cumings; Mrs. John Bradley (Florence Briggs Th...,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Heikkinen; Miss. Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Futrelle; Mrs. Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,Allen; Mr. William Henry,male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
pclass = titanic['Pclass'].tolist()
survived = titanic['Survived'].tolist()

In [36]:
# строим таблицу сопряженности
contingency_table = pd.crosstab(pclass, survived)
contingency_table

col_0,0,1
row_0,,
1,80,136
2,97,87
3,372,119


In [37]:
from scipy.stats import chi2_contingency

In [38]:
chi2_contingency(contingency_table)

Chi2ContingencyResult(statistic=102.88898875696056, pvalue=4.549251711298793e-23, dof=2, expected_freq=array([[133.09090909,  82.90909091],
       [113.37373737,  70.62626263],
       [302.53535354, 188.46464646]]))

In [40]:
# pvalue намного меньше 0,05. Значит, класс влияет на шанс выжить
# пассажиры 1-го класса имели больше шансов выжить, чем пассажиры 2-го и 3-го классов